Risk
====

In [1]:
from collections import namedtuple
import data as datalib
import puma as tw

from tomahawk import risk

In [2]:
# setup the market data
db_credentials = credentials('test', 'localhost', prefix='db_')
csvdf = datalib.CsvDataFeed("c:/rmbaries/git/pifq/src/puma/data/tests/inst/csv_data_feed", source_name='test_csv_data_feed')
csvdh = datalib.HistoricalDataManager(csvdf, **db_credentials)
csvdl = datalib.LiveDataManager(csvdf, **db_credentials)
mdm = datalib.MarketDataManager(csvdh, csvdl)

In [3]:
# setup the Strategy, Portfolio and PositionManager
oms = tw.OrderManager('test_unit', None)
pm = tw.PositionManager('test_unit', oms, None)
port = tw.Portfolio('port_test', oms, pm)
obj_bridge = namedtuple('OB', 'order_manager, portfolio, market_data_manager')(oms, port, mdm)
strat = tw.strategy.ExampleStrategy('TEST1', obj_bridge)
port.add_strategy(strat)

In [4]:
# initialize Risk

rk = risk.Risk(oms)

In [5]:
# insert symbols into strategy
strat.add_symbols([('stock', 'test.sym.1', '1min'), ('stock', 'test.sym.2', '1min')])

In [6]:
# create some orders and have Portfolio process them

oms.market_state('stock', True)
strat.order('stock', 'test.sym.1', 'B', 1000, 'LIMIT', 100.5)
strat.order('stock', 'test.sym.2', 'S', 55, 'LIMIT', 5.5)
port.process_orders()

# see the orders in OrderManager
oms.open_orders_df().show()

In [7]:
# process the orders in the staged order queue of the Portfolio

rk.process_portfolio_orders(port)

In [8]:
# see the state of the orders

oms.open_orders_df().show()

In [9]:
# and closed orders

oms.closed_orders_df().show()

In [11]:
# print the first accepted order
oms.orders_list({'state': 'RISK_ACCEPTED'})[0].print()

In [12]:
# print the first rejected order
oms.orders_list({'state': 'RISK_REJECTED'})[0].print()